In [ ]:
import pandas as pd

import torch

from tqdm import tqdm

tqdm.pandas()

import warnings

warnings.filterwarnings("ignore")

import sklearn
import seaborn as sns

In [ ]:
df_with_unsup = pd.read_csv("unsupo_train.csv")
df_test =pd.read_csv("manually_test.csv")

In [ ]:
df_test = df_test.rename(columns={'sentiment_final': 'sentiment'})

# 임베딩 모델링

In [ ]:
train_with_unsup = tuple(df_with_unsup)

In [ ]:
from sklearn.model_selection import train_test_split

train_with_unsup = df_with_unsup
test_with_unsup = df_test


print("With Unsupervised Data:")
print("Train size:", len(train_with_unsup))
print("Test size:", len(test_with_unsup))

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer,RobertaTokenizer

class SentimentDataset(Dataset):    
    def __init__(self, dataframe, max_length):     
        self.data = dataframe       
        #self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # 
        #self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased') # 
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.max_length = max_length    # max_len

    def __len__(self):
        return len(self.data)           
 
    def __getitem__(self, index): 
        text = str(self.data.loc[index, 'content'])
        label = self.data.loc[index, 'sentiment']
        """
        input_ids: 토큰화된 텍스트의 정수 시퀀스로 이루어진 리스트입니다. BERT 모델의 입력으로 사용됩니다.
        token_type_ids: 각 토큰의 문장 유형을 나타내는 리스트입니다. 문장이 두 개일 경우 첫 번째 문장의 토큰은 0으로, 두 번째 문장의 토큰은 1로 설정됩니다.
        attention_mask: 어텐션 마스크로, 입력 토큰 중에서 실제 단어에 해당하는 부분은 1, 패딩 토큰에 해당하는 부분은 0으로 설정됩니다.
        overflowing_tokens: 인코딩 과정에서 잘린 토큰이 있는 경우 해당 토큰을 포함합니다.
        num_truncated_tokens: 인코딩 과정에서 잘린 토큰의 수를 나타냅니다.
        special_tokens_mask: 특수 토큰의 위치를 나타내는 리스트입니다. 특수 토큰은 [CLS], [SEP], [PAD] 등을 말합니다.
        """
        inputs = self.tokenizer.encode_plus(        
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze(0).to(device)  # (seq_length,)
        attention_mask = inputs['attention_mask'].squeeze(0).to(device)  # (seq_length,)
        label = torch.tensor(label).to(device)

        return input_ids, attention_mask, label


def create_data_loader(dataframe, max_length, batch_size):
    dataset = SentimentDataset(dataframe, max_length)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True
    )


max_length = 64 # 32,128,256
batch_size = 64   #8,16,32,64,128


train_loader = create_data_loader(train_with_unsup, max_length, batch_size)
test_loader = create_data_loader(test_with_unsup, max_length, batch_size)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import classification_report


#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device) 
#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device) 
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

 
model.train()
for epoch in range(5):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 99: 
            print(f"Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss / 100}")
            running_loss = 0.0
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")


model.eval()
predictions, true_labels = [], []
for batch in test_loader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    _, predicted = torch.max(logits, 1)
    predictions.extend(predicted.tolist())
    true_labels.extend(labels.tolist())


accuracy = accuracy_score(true_labels, predictions)
print( accuracy)

print(classification_report(true_labels, predictions, target_names=['neg', 'pos'],digits=4))